
# Langgraph with AgentNeo Integration
 This Jupyter notebook demonstrates the integration of AgentNeo, a powerful tracing and monitoring tool, with Langgraph, a graph-based approach to managing language models with an agent-based system to enhance the automation and decision-making capabilities of your application. This integration allows for comprehensive analysis and debugging of AI-powered systems.


 # Setup and Imports
First, let's import the necessary libraries and set up our environment.

In [ ]:
%pip install agentneo==1.1.1

In [2]:
%%capture --no-stderr
%pip install -U langchain langchain_openai langsmith pandas langchain_experimental matplotlib langgraph langchain_core

In [ ]:
%pip install lite-llm

In [4]:
import getpass
import os
import openai
from litellm import completion
from typing import List, Dict, Any
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel

# Function for the OpenAI API key if it's not already set in the environment
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")


 # Initialize AgentNeo Session and Tracer
 Now, let's set up our AgentNeo session and tracer.



In [ ]:

# Import AgentNeo and Setup Tracing
from agentneo import AgentNeo, Tracer, Execution, launch_dashboard 

# Initialize AgentNeo session
neo_session = AgentNeo(session_name="langGraph_example1")
try:
    neo_session.create_project(project_name="LangGraph_with_AgentNeo2 ")
except:
    neo_session.connect_project(project_name="LangGraph_with_AgentNeo2")

# Create tracer
tracer = Tracer(session=neo_session)

tracer.start()


 # Define Agents and Tools
Now, let's create our AI tools using langgraph.

In [8]:
# Create a standalone PromptInstructions class
@tracer.trace_agent(name="PromptInstructions")
class PromptInstructions(BaseModel):
    """Instructions on how to prompt the LLM."""

    objective: str
    variables: List[str]
    constraints: List[str]
    requirements: List[str]



In [17]:
# Create a class for managing prompt generation
@tracer.trace_agent(name="PromptGenerator")
class PromptGenerator:
    def __init__(self):
        self.template = """Your job is to get information from a user about what type of prompt template they want to create.

        You should get the following information from them:

        - What the objective of the prompt is
        - What variables will be passed into the prompt template
        - Any constraints for what the output should NOT do
        - Any requirements that the output MUST adhere to

        If you are not able to discern this info, ask them to clarify! Do not attempt to wildly guess.

        After you are able to discern all the information, call the relevant tool."""
        
        self.llm = ChatOpenAI(temperature=0.5)
        self.llm_with_tool = self.llm.bind_tools([PromptInstructions])

    @tracer.trace_tool(name="get message")
    def get_messages_info(self, messages):
        return [SystemMessage(content=self.template)] + messages

    @tracer.trace_tool(name="info chain")
    def info_chain(self, state):
        messages = self.get_messages_info(state["messages"])
        response = self.llm_with_tool.invoke(messages)
        return {"messages": [response]}

    @tracer.trace_tool(name="get prompt")
    def get_prompt_messages(self, messages: list):
        tool_call = None
        other_msgs = []
        for m in messages:
            if isinstance(m, AIMessage) and m.tool_calls:
                tool_call = m.tool_calls[0]["args"]
            elif isinstance(m, ToolMessage):
                continue
            elif tool_call is not None:
                other_msgs.append(m)
        prompt_system = """Based on the following requirements, write a good prompt template:
        
        {reqs}"""
        return [SystemMessage(content=prompt_system.format(reqs=tool_call))] + other_msgs

    @tracer.trace_tool(name="prompt gen chain")
    def prompt_gen_chain(self, state):
        messages = self.get_prompt_messages(state["messages"])
        response = self.llm.invoke(messages)
        return {"messages": [response]}

In [ ]:
# Instantiate the PromptGenerator class
prompt_generator = PromptGenerator()

In [24]:
# Importing the `Literal` type from the `typing` module, which allows defining specific literal values for type hints.

from typing import Literal
# Importing END which likely represents the end or termination of a process or a node
from langgraph.graph import END

@tracer.trace_tool(name="get state")
def get_state(state):
    messages = state["messages"]
    if isinstance(messages[-1], AIMessage) and messages[-1].tool_calls:
        return "add_tool_message"
    elif not isinstance(messages[-1], HumanMessage):
        return END
    return "info"


# # Importing necessary components 
managing state graphs, saving memory, handling messages, and typing annotations.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict

@tracer.trace_agent(name="workflow")
class State(TypedDict):
    messages: Annotated[list, add_messages]

memory = MemorySaver()
workflow = StateGraph(State)

# Instantiate the PromptGenerator class
prompt_generator = PromptGenerator()

workflow.add_node("info", prompt_generator.info_chain)
workflow.add_node("prompt", prompt_generator.prompt_gen_chain)

@workflow.add_node
def add_tool_message(state: State):
    return {
        "messages": [
            ToolMessage(
                content="Prompt generated!",
                tool_call_id=state["messages"][-1].tool_calls[0]["id"],
            )
        ]
    }

workflow.add_conditional_edges("info", get_state, ["add_tool_message", "info", END])
workflow.add_edge("add_tool_message", "prompt")
workflow.add_edge("prompt", END)
workflow.add_edge(START, "info")
graph = workflow.compile(checkpointer=memory)


# Displaying a PNG image of a graph generated from a LangGraph instance using Mermaid visualization

In [ ]:
# %% 
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))


 # Continuously prompts the user for input
  streams responses from the graph, and handles fallback cached responses in case of input failure.

In [ ]:

# %% 
import uuid

cached_human_responses = ["hi!", "rag prompt", "1 rag, 2 none, 3 no, 4 no", "red", "q"]
cached_response_index = 0
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

while True:
    try:
        user = input("User (q/Q to quit): ")
    except:
        user = cached_human_responses[cached_response_index]
        cached_response_index += 1
    print(f"User (q/Q to quit): {user}")
    
    if user in {"q", "Q"}:
        print("AI: Byebye")
        break
        
    output = None
    for output in graph.stream(
        {"messages": [HumanMessage(content=user)]}, config=config, stream_mode="updates"
    ):
        last_message = next(iter(output.values()))["messages"][-1]
        last_message.pretty_print()

    if output and "prompt" in output:
        print("Done!")

# %% 
# Stop Tracing and Launch Dashboard
tracer.stop()
print(tracer.trace_id)


 # Metrics Evaluation
 Supported Metrics Goal Decomposition Efficiency (goal_decomposition_efficiency) Goal Fulfillment Rate (goal_fulfillment_rate) Tool Correctness Metric (tool_correctness_metric) Tool Call Success Rate Metric (tool_call_success_rate_metric)


In [ ]:
# %% 
# Execute metrics
from agentneo import Execution
exe = Execution(session=neo_session, trace_id=tracer.trace_id)  
exe.execute(metric_list=['goal_decomposition_efficiency', 'goal_fulfillment_rate', 'tool_call_success_rate_metric'])
metric_results = exe.get_results()
print(metric_results) 
metric_results

In [ ]:
# %% 
# Launching dashboard
neo_session.launch_dashboard()

# %%q
